### Shooting Data Prediction

> <sub>⚠️ **Note**: Internal links (like Table of Contents) work best when this notebook is opened in **Jupyter Notebook** or **nbviewer.org**.<br>
> GitHub does **not support scrolling to sections** inside `.ipynb` files.</sub>

---
######  - [Reading Shooting Dataset](#Reading-Shooting-Dataset)
######  - [Scaling the facilities count columns](#Scaling-the-facilities-count-columns)
######  - [Modelling with 2 Clusters](#Modelling-with-2-Clusters)
######  - [Running ANN Model](#Running-ANN-Model)
######  - [Running Random Forest Model](#Running-Random-Forest-Model)
---

#### Reading Shooting Dataset
######  - [_Click here to move back to index_](#Shooting-Data-Prediction)

In [5]:
import sys
sys.path.append('src') 


In [6]:
from Data_Preprocessing import load_data

df_new = load_data('data/processed/Shooting_Data_For_Prediction.csv')

df_new.head(5)


,year_month,BORO,PRECINCT,JURISDICTION_CODE,After_6PM_Flag,Times Square Distance,Grand Central Distance,Murder_Flag,More_Than_25_Years,month,...,"EDUCATION, CHILD WELFARE, AND YOUTH",HEALTH AND HUMAN SERVICES,LIBRARIES AND CULTURAL PROGRAMS,"PARKS, GARDENS, AND HISTORICAL SITES","PUBLIC SAFETY, EMERGENCY SERVICES, AND ADMINISTRATION OF JUSTICE",Boro_Key,Cluster_3,Cluster_2,Average of Latitude,Average of Longitude
0,2010-01,BRONX,40,0,1.0,7.633244,7.700929,0.0,1.0,1,...,394,191,8,59,11,B,1,1,40.806578,-73.919949
1,2010-01,BRONX,40,2,1.0,7.696431,7.743550,0.0,1.0,1,...,394,191,8,59,11,B,2,0,40.805900,-73.918100
2,2010-01,BRONX,41,0,1.0,10.402580,10.356822,0.0,1.0,1,...,145,82,5,36,5,B,1,1,40.816673,-73.887998
3,2010-01,BRONX,42,0,1.0,10.617731,10.781218,0.0,1.0,1,...,304,106,5,51,7,B,1,1,40.832897,-73.905479
4,2010-01,BRONX,42,2,0.5,10.937563,11.133255,0.0,0.5,1,...,304,106,5,51,7,B,0,0,40.837303,-73.906738


In [7]:
df_new.describe()

,PRECINCT,JURISDICTION_CODE,After_6PM_Flag,Times Square Distance,Grand Central Distance,Murder_Flag,More_Than_25_Years,month,New Year = 1,Christmas = 12,...,CORE INFRASTRUCTURE AND TRANSPORTATION,"EDUCATION, CHILD WELFARE, AND YOUTH",HEALTH AND HUMAN SERVICES,LIBRARIES AND CULTURAL PROGRAMS,"PARKS, GARDENS, AND HISTORICAL SITES","PUBLIC SAFETY, EMERGENCY SERVICES, AND ADMINISTRATION OF JUSTICE",Cluster_3,Cluster_2,Average of Latitude,Average of Longitude
count,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,...,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000,8346.000000
mean,64.973281,0.458303,0.734703,11.793416,11.409810,0.176552,0.734703,6.633956,0.076803,0.080158,...,69.199736,231.099808,85.052960,19.838725,43.050204,5.991014,1.173616,0.559190,40.737790,-73.915881
std,29.714550,0.836838,0.377192,5.091462,4.978259,0.312586,0.377192,3.334637,0.266295,0.271554,...,34.805948,75.752981,35.635719,20.187387,20.778082,2.747892,0.854792,0.496514,0.086009,0.068871
min,1.000000,0.000000,0.000000,0.027750,0.170192,0.000000,0.000000,1.000000,0.000000,0.000000,...,2.000000,1.000000,0.000000,4.000000,3.000000,1.000000,0.000000,0.000000,40.516572,-74.230149
25%,42.000000,0.000000,0.500000,8.184671,7.797266,0.000000,0.500000,4.000000,0.000000,0.000000,...,42.000000,168.000000,57.000000,8.000000,30.000000,4.000000,0.000000,0.000000,40.670527,-73.947701
50%,67.000000,0.000000,1.000000,11.270932,10.876782,0.000000,1.000000,7.000000,0.000000,0.000000,...,66.000000,229.000000,83.000000,15.000000,38.000000,5.000000,1.000000,1.000000,40.711357,-73.921403
75%,83.000000,0.000000,1.000000,14.786510,14.297525,0.272727,1.000000,9.000000,0.000000,0.000000,...,89.000000,277.000000,105.000000,23.000000,51.000000,7.000000,2.000000,1.000000,40.820108,-73.886222
max,123.000000,2.000000,1.000000,33.655187,33.527926,1.000000,1.000000,12.000000,1.000000,1.000000,...,352.000000,453.000000,191.000000,149.000000,182.000000,18.000000,2.000000,1.000000,40.910818,-73.718033


#### Scaling the facilities count columns
######  - [_Click here to move back to index_](#Shooting-Data-Prediction)

In [9]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()

columns = ['ADMINISTRATION OF GOVERNMENT',
       'CORE INFRASTRUCTURE AND TRANSPORTATION', 'EDUCATION, CHILD WELFARE, AND YOUTH', 'HEALTH AND HUMAN SERVICES',
       'LIBRARIES AND CULTURAL PROGRAMS', 'PARKS, GARDENS, AND HISTORICAL SITES','PUBLIC SAFETY, EMERGENCY SERVICES, AND ADMINISTRATION OF JUSTICE']

df_new[columns] = pd.DataFrame(scaler.fit_transform(df_new[columns]))
df_new.head(5)


,year_month,BORO,PRECINCT,JURISDICTION_CODE,After_6PM_Flag,Times Square Distance,Grand Central Distance,Murder_Flag,More_Than_25_Years,month,...,"EDUCATION, CHILD WELFARE, AND YOUTH",HEALTH AND HUMAN SERVICES,LIBRARIES AND CULTURAL PROGRAMS,"PARKS, GARDENS, AND HISTORICAL SITES","PUBLIC SAFETY, EMERGENCY SERVICES, AND ADMINISTRATION OF JUSTICE",Boro_Key,Cluster_3,Cluster_2,Average of Latitude,Average of Longitude
0,2010-01,BRONX,40,0,1.0,7.633244,7.700929,0.0,1.0,1,...,0.869469,1.000000,0.027586,0.312849,0.588235,B,1,1,40.806578,-73.919949
1,2010-01,BRONX,40,2,1.0,7.696431,7.743550,0.0,1.0,1,...,0.869469,1.000000,0.027586,0.312849,0.588235,B,2,0,40.805900,-73.918100
2,2010-01,BRONX,41,0,1.0,10.402580,10.356822,0.0,1.0,1,...,0.318584,0.429319,0.006897,0.184358,0.235294,B,1,1,40.816673,-73.887998
3,2010-01,BRONX,42,0,1.0,10.617731,10.781218,0.0,1.0,1,...,0.670354,0.554974,0.006897,0.268156,0.352941,B,1,1,40.832897,-73.905479
4,2010-01,BRONX,42,2,0.5,10.937563,11.133255,0.0,0.5,1,...,0.670354,0.554974,0.006897,0.268156,0.352941,B,0,0,40.837303,-73.906738


#### Modelling with 2 Clusters
######  - [_Click here to move back to index_](#Shooting-Data-Prediction)

In [11]:
# Filter on followng features

features = [
     'BORO'
    , 'PRECINCT'
    , 'JURISDICTION_CODE'
    , 'After_6PM_Flag'
   ,'Times Square Distance'
#    , 'Grand Central Distance'
    , 'Murder_Flag'
    ,'More_Than_25_Years'
#   , 'month'
   , 'seasonality'
   , 'New Year = 1'
   ,'Christmas = 12'
   , 'Thanksgivigiving = 11'
  , 'Indep. Day = 7'
    ,'Halloween = 10'
   , 'year'
    , 'Unemployment_Flag'
#    , 'Year_month_temperature'
   ,'Avg Temp (>=70 F) Flag'
    , 'Avg Temp (<40 F) Flag'
    # ,'ADMINISTRATION OF GOVERNMENT'
    ,'CORE INFRASTRUCTURE AND TRANSPORTATION'
    # ,'EDUCATION, CHILD WELFARE, AND YOUTH', 'HEALTH AND HUMAN SERVICES'
    # ,'LIBRARIES AND CULTURAL PROGRAMS', 'PARKS, GARDENS, AND HISTORICAL SITES'
   ,'PUBLIC SAFETY, EMERGENCY SERVICES, AND ADMINISTRATION OF JUSTICE'
     ]


#### Running ANN Model
######  - [_Click here to move back to index_](#Shooting-Data-Prediction)

In [13]:
from ML_Utilities import train_ann_with_outputs, run_ann_gridsearch_and_save_outputs

results_df, best_model_output = run_ann_gridsearch_and_save_outputs(
    df=df_new,
    features=features,
    target_col='Cluster_2',
    output_dir='output',
    max_iters=[1000, 2000, 3000],
    batch_sizes=[64, 128, 256],
    solvers=['adam', 'lbfgs', 'sgd']
)


Training ANN Models:   4%|██▎                                                           | 1/27 [00:00<00:24,  1.08it/s]C:\Users\utkar\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
Training ANN Models:  15%|█████████▏                                                    | 4/27 [00:25<02:04,  5.41s/it]C:\Users\utkar\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_it

#### Running Random Forest Model
######  - [_Click here to move back to index_](#Shooting-Data-Prediction)

In [15]:
from ML_Utilities import train_rf_with_outputs, run_rf_gridsearch_and_save_outputs

results_df, best_model_output = run_rf_gridsearch_and_save_outputs(
    df=df_new,
    features=features,
    target_col='Cluster_2',
    output_dir='output',
    n_estimators_list=[100, 200, 300],
    max_depth_list=[10, 15, 20],
    min_samples_split_list=[2, 5, 10]
)


Training Random Forest Models: 100%|███████████████████████████████████████████████████| 27/27 [01:05<00:00,  2.44s/it]
